In [99]:
# Importing Libraries
from collections import defaultdict
from pathlib import Path
import numpy as np
import pandas as pd
from collections import namedtuple
from tabulate import tabulate

import torch
import os

from adversarial_debiasing import AdversarialDebiasing
from load_data import load_data, transform_data, Datapoint

from load_vectors import load_pretrained_vectors, load_vectors
import config
import utility_functions

In [100]:
# For autoreloading changes made in other python scripts
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_extautoreload


In [101]:
# Loading the word vectors dictionary
word_vectors = load_pretrained_vectors(config.glove_embedding_data_path, config.save_dir, config.glove_save_file, config.use_glove)

Loading from saved file.


In [102]:
# Load the google analogies training dataset:
analogy_dataset = load_data()
analogy_dataset

task='capital-world'),
 Raw_Datapoint(x1='Ashgabat', x2='Turkmenistan', x3='Banjul', y='Gambia', task='capital-world'),
 Raw_Datapoint(x1='Ashgabat', x2='Turkmenistan', x3='Beijing', y='China', task='capital-world'),
 Raw_Datapoint(x1='Ashgabat', x2='Turkmenistan', x3='Beirut', y='Lebanon', task='capital-world'),
 Raw_Datapoint(x1='Ashgabat', x2='Turkmenistan', x3='Belgrade', y='Serbia', task='capital-world'),
 Raw_Datapoint(x1='Ashgabat', x2='Turkmenistan', x3='Belmopan', y='Belize', task='capital-world'),
 Raw_Datapoint(x1='Ashgabat', x2='Turkmenistan', x3='Berlin', y='Germany', task='capital-world'),
 Raw_Datapoint(x1='Ashgabat', x2='Turkmenistan', x3='Bern', y='Switzerland', task='capital-world'),
 Raw_Datapoint(x1='Ashgabat', x2='Turkmenistan', x3='Bishkek', y='Kyrgyzstan', task='capital-world'),
 Raw_Datapoint(x1='Ashgabat', x2='Turkmenistan', x3='Bratislava', y='Slovakia', task='capital-world'),
 Raw_Datapoint(x1='Ashgabat', x2='Turkmenistan', x3='Brussels', y='Belgium', task='c

In [103]:
# Transform the data such that it includes the embeddings of the words in consideration
transformed_analogy_dataset = transform_data(word_vectors, analogy_dataset)

In [104]:
# Testing the transformed analogy dataset
print(transformed_analogy_dataset[0].analogy_embeddings.shape)
print(transformed_analogy_dataset[0].gt_embedding.shape)
print(transformed_analogy_dataset[0].protected.shape)

(300,)
(100,)
(1,)


In [105]:
# Training the variant of the model without debiasing
non_debiased_model = AdversarialDebiasing(debias=False)
non_debiased_model.fit(dataset=transformed_analogy_dataset)

[0/50] Running epoch
epoch 0; iter: 0; batch classifier loss: 0.357857
[1/50] Running epoch
epoch 1; iter: 0; batch classifier loss: 0.346382
[2/50] Running epoch
epoch 2; iter: 0; batch classifier loss: 0.331825
[3/50] Running epoch
epoch 3; iter: 0; batch classifier loss: 0.354570
[4/50] Running epoch
epoch 4; iter: 0; batch classifier loss: 0.336471
[5/50] Running epoch
epoch 5; iter: 0; batch classifier loss: 0.341985
[6/50] Running epoch
epoch 6; iter: 0; batch classifier loss: 0.331365
[7/50] Running epoch
epoch 7; iter: 0; batch classifier loss: 0.325093
[8/50] Running epoch
epoch 8; iter: 0; batch classifier loss: 0.355655
[9/50] Running epoch
epoch 9; iter: 0; batch classifier loss: 0.323041
[10/50] Running epoch
epoch 10; iter: 0; batch classifier loss: 0.344972
[11/50] Running epoch
epoch 11; iter: 0; batch classifier loss: 0.309037
[12/50] Running epoch
epoch 12; iter: 0; batch classifier loss: 0.328624
[13/50] Running epoch
epoch 13; iter: 0; batch classifier loss: 0.34429

In [106]:
# Training the variant of the model with debiasing
debiased_model = AdversarialDebiasing()
debiased_model.fit(dataset=transformed_analogy_dataset)

poch 34; iter: 120; batch classifier loss: 0.335279; batch adversarial loss: 0.182830
epoch 34; iter: 130; batch classifier loss: 0.349417; batch adversarial loss: 0.193133
epoch 34; iter: 140; batch classifier loss: 0.350720; batch adversarial loss: 0.228916
[35/50] Running epoch
epoch 35; iter: 0; batch classifier loss: 0.300946; batch adversarial loss: 0.202169
epoch 35; iter: 10; batch classifier loss: 0.327983; batch adversarial loss: 0.159899
epoch 35; iter: 20; batch classifier loss: 0.311907; batch adversarial loss: 0.196383
epoch 35; iter: 30; batch classifier loss: 0.339024; batch adversarial loss: 0.171867
epoch 35; iter: 40; batch classifier loss: 0.348780; batch adversarial loss: 0.236129
epoch 35; iter: 50; batch classifier loss: 0.343712; batch adversarial loss: 0.222972
epoch 35; iter: 60; batch classifier loss: 0.357575; batch adversarial loss: 0.185604
epoch 35; iter: 70; batch classifier loss: 0.365883; batch adversarial loss: 0.227271
epoch 35; iter: 80; batch class

In [107]:
# Examples to test the models upon
datapoints, test_analogies = [], []
with open(os.path.join('data', 'sexism-traps.txt'), 'r') as f:
    # Reading each line
    for line in f.readlines():
        words = line.split()
        if words[0] == ':':
            continue
        test_analogies.append(words)
        word_embeddings = word_vectors[words]
        word_embeddings = np.reshape(word_embeddings, (1, -1))
        datapoints.append(word_embeddings)
datapoints = np.vstack(datapoints)
print(datapoints.shape)

(20, 300)


In [108]:
# Qualitative evaluation of the non-debiased model
non_debiased_predictions = non_debiased_model.predict(datapoints)
non_debiased_most_similar_list = utility_functions.obtain_most_similar(non_debiased_predictions, word_vectors)

In [109]:
# Displaying the similarity list for the non-debiased model
non_debiased_most_similar_list_data_frames = []
for i in range(len(non_debiased_most_similar_list)):
    # print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    temp_data_frame = pd.DataFrame(non_debiased_most_similar_list[i][1:], columns = ['Neighbor', 'Similarity'])
    non_debiased_most_similar_list_data_frames.append(temp_data_frame)
    # print(tabulate(temp_data_frame, headers='keys', tablefmt='psql', showindex=False))

In [110]:
# Qualitative evaluation of the debiased model
debiased_predictions = debiased_model.predict(datapoints)
debiased_most_similar_list = utility_functions.obtain_most_similar(debiased_predictions, word_vectors)

In [111]:
# Displaying the similarity list for the debiased model
debiased_most_similar_list_data_frames = []
for i in range(len(debiased_most_similar_list)):
    # print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    temp_data_frame = pd.DataFrame(debiased_most_similar_list[i][1:], columns = ['Neighbor', 'Similarity'])
    debiased_most_similar_list_data_frames.append(temp_data_frame)
    # print(tabulate(temp_data_frame, headers='keys', tablefmt='psql', showindex=False))

In [112]:
# Combining the dataframes pertaining to both the variants of the model
iterables = [['Biased', 'Debiased'], ['Neighbour', 'Similarity']]
index = pd.MultiIndex.from_product(iterables)
overall_data_frames_list = []
for i in range(len(non_debiased_most_similar_list)):
    overall_list = []
    print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    for j in range(len(non_debiased_most_similar_list[i][1:])):
        temp_list = []
        temp_list.append(non_debiased_most_similar_list[i][j][0])
        temp_list.append(round(non_debiased_most_similar_list[i][j][1], 3))
        temp_list.append(debiased_most_similar_list[i][j][0])
        temp_list.append(round(debiased_most_similar_list[i][j][1], 3))
        overall_list.append(temp_list)
    temp_df = pd.DataFrame(overall_list, columns = index)
    # print(temp_df.to_string(index = False))
    print(tabulate(temp_df, headers = ['Biased\nNeighbour', 'Biased\nSimilarity', 'Debiased\nNeighbour', 'Debiased\nSimilarity'], tablefmt = 'psql', showindex = False))
    overall_data_frames_list.append(temp_df)

he : strong :: she :
+-------------+--------------+-------------+--------------+
| Biased      |       Biased | Debiased    |     Debiased |
| Neighbour   |   Similarity | Neighbour   |   Similarity |
|-------------+--------------+-------------+--------------|
| strong      |        0.826 | strong      |        0.785 |
| heart       |        0.706 | heart       |        0.728 |
| stay        |        0.695 | always      |        0.684 |
| keep        |        0.679 | girl        |        0.681 |
| calm        |        0.678 | keep        |        0.661 |
| girl        |        0.668 | she         |        0.651 |
| always      |        0.666 | your        |        0.649 |
| loving      |        0.648 | stay        |        0.645 |
| 're         |        0.644 | really      |        0.644 |
+-------------+--------------+-------------+--------------+
he : boss :: she :
+-------------+--------------+-------------+--------------+
| Biased      |       Biased | Debiased    |     Debiased |


In [113]:
# Fake dataset for testing purposes

# embedding_dim = 100
# analogy_dataset = [
#     Datapoint(
#     analogy_embeddings=np.random.normal(0, 1, size=(3 * embedding_dim, 1)), 
#     gt_embedding=np.random.normal(0, 1, size=(embedding_dim, 1)),
#     protected_embedding=np.random.uniform(0, 1, size=(1))) for n in range(0, 1000)
# ]